# Test automatic reflector detection

In [1]:
import os
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

data_folder = '/Users/rdcrlrka/Research/Soo_locks'
video_folder = os.path.join(data_folder, '20251120_video')
out_folder = os.path.join(data_folder, '20251120_imagery')
os.makedirs(out_folder, exist_ok=True)

# import utility functions
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))
import ortho_utils

In [2]:
# Extract images from videos
video_files = sorted(
    glob(os.path.join(video_folder, 'dac1', '*.avi')) +
    glob(os.path.join(video_folder, 'dac2', '*.avi')) + 
    glob(os.path.join(video_folder, 'dac3', '*.avi'))
    )

ortho_utils.process_video_files(
    video_files, 
    target_time_string = "20251120140600",
    output_folder = out_folder
    )

Target time: 2025-11-20 14:06:00

Processing /Users/rdcrlrka/Research/Soo_locks/20251120_video/dac1/N910A6_ch1_main_20251120140500_20251120141000.avi
Detected video time range: 2025-11-20 14:05:00 to 2025-11-20 14:10:00
Extracted frame -> /Users/rdcrlrka/Research/Soo_locks/20251120_imagery/ch01_20251120140600.tiff

Processing /Users/rdcrlrka/Research/Soo_locks/20251120_video/dac1/N910A6_ch2_main_20251120140500_20251120141000.avi
Detected video time range: 2025-11-20 14:05:00 to 2025-11-20 14:10:00
Extracted frame -> /Users/rdcrlrka/Research/Soo_locks/20251120_imagery/ch02_20251120140600.tiff

Processing /Users/rdcrlrka/Research/Soo_locks/20251120_video/dac1/N910A6_ch3_main_20251120140500_20251120141000.avi
Detected video time range: 2025-11-20 14:05:00 to 2025-11-20 14:10:00
Extracted frame -> /Users/rdcrlrka/Research/Soo_locks/20251120_imagery/ch03_20251120140600.tiff

Processing /Users/rdcrlrka/Research/Soo_locks/20251120_video/dac1/N910A6_ch4_main_20251120140500_20251120141000.avi
D

## Test blob detection

In [ ]:
def preprocess_image(img, threshold=None, blur=True):
    # Pre-blur before thresholding to help smooth edges
    if blur:
        img = cv2.GaussianBlur(img, (5, 5), 0)

    # Better thresholding
    if threshold=='otsu':
        _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    elif (type(threshold)==int) | (type(threshold)==float):
        _, binary = cv2.threshold(img, threshold, threshold, cv2.THRESH_BINARY)
    else:
        binary = img

    # Fill gaps in edges (helps circularity)
    kernel = np.ones((3,3), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=2)

    return binary


def detect_circular_contour(patch):
    img = patch.copy()
    
    # Find contours
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not contours:
        print('No contours found.')
        return None
    
    # Remove border-connected components
    def contour_touches_border(contour, w, h):
        for pt in contour:
            x, y = pt[0]
            if x == 0 or x == w-1 or y == 0 or y == h-1:
                return True
        return False
    h, w = img.shape
    contours = [
        c for c in contours
        if not contour_touches_border(c, w, h)
    ]

    if len(contours) == 0:
        print('No contours not touching image border.')
        return None

    # Filter by minimal area
    contours = [c for c in contours if cv2.contourArea(c) > 20]
    if not contours:
        print('No contours found with area > 20.')
        return None

    # Circularity test
    circ_contours = []
    for c in contours:
        area = cv2.contourArea(c)
        peri = cv2.arcLength(c, True)
        if peri == 0:
            continue
        circ = 4 * np.pi * area / (peri**2)
        if circ > 0.55:
            circ_contours.append(c)
    if not circ_contours:
        print('No contours found with circularity > 0.55.')
        return None
    
    return circ_contours


def match_reflectors(image1, image2, refl_brightness='low', blur=False):
    # determine threshold
    if refl_brightness=='low':
        threshold = 50
    elif refl_brightness=='med':
        threshold = 100
    elif refl_brightness=='high':
        threshold = 150
    else:
        raise ValueError(f"Variable refl_brightness not recognized. Check inputs.")

    # preprocess images
    img1 = image1.copy()
    img1 = preprocess_image(img1, threshold, blur)
    img2 = image2.copy()
    img2 = preprocess_image(img2, threshold, blur)

    # Identify circular contours
    ct1 = detect_circular_contour(img1)
    ct2 = detect_circular_contour(img2)

    if (ct1 == None) | (ct2 == None):
        return None, None, None, None

    # Use longest contour in img1 as the template
    ct1_template = max(ct1, key=lambda c: cv2.contourArea(c))

    # Match contour in the second image
    for c in ct2:
        match = cv2.matchShapes(ct1_template,c,3,0.0)
        #valid matches would be less than 0.15
        if match < 0.15:
            ct2_match = c
        else:
            print('No match found.')
            return None, None, None, None

    # Get center coordinates
    def get_object_center(ct):
        M = cv2.moments(ct)
        if M['m00'] == 0:
            return None
        cx = M['m10'] / M['m00']
        cy = M['m01'] / M['m00']
        return cx, cy
    ct1_center = get_object_center(ct1_template)
    ct2_center = get_object_center(ct2_match)
    
    # Reformat arrays
    ct1 = np.array([x[0] for x in ct1_template])
    ct2 = np.array([x[0] for x in ct2_match])

    return ct1, ct1_center, ct2, ct2_center


# open images
ch = 'ch11'
image1_files = sorted(glob(os.path.join(out_folder, '*.tiff')))
image1_file = [x for x in image1_files if ch in os.path.basename(x)][0]
image1 = cv2.imread(image1_file, cv2.IMREAD_GRAYSCALE)
image2_files = sorted(glob(os.path.join(data_folder, '..', 'camera_calibration', 'images20251021', '*.tiff')))
image2_file = [x for x in image2_files if ch in os.path.basename(x)][0]
image2 = cv2.imread(image2_file, cv2.IMREAD_GRAYSCALE)

# ch10
if ch=='ch10':
    pts = [
        (987,993),
        (1121,40),
        (2767,844),
        (2754,1646)
    ]
    refl_brightness = [
        'high', 'high', 'med', 'med'
    ]
elif ch=='ch11':
    pts = [
        (2482,570),
        # (2502,1788),
        (714,797),
        # (1345,1576),
        # (824,2129)
    ]
    refl_brightness = [
        'low', 'high', 
    ]

for i, pt in enumerate(tqdm(pts)):
    # crop images
    h,w=image1.shape
    xlim = np.clip([pt[0]-100, pt[0]+100], 0, w-1)
    ylim = np.clip([pt[1]-100, pt[1]+100], 0, h-1)
    image1_crop = image1[ylim[0]:ylim[1], xlim[0]:xlim[1]]
    image2_crop = image2[ylim[0]:ylim[1], xlim[0]:xlim[1]]

    if refl_brightness[i]=='low':
        threshold = 50 
    elif refl_brightness[i]=='med':
        threshold = 100
    else: 
        threshold = 150

    fig, ax = plt.subplots(1, 2, figsize=(10,5))
    ax[0].imshow(image1_crop > threshold, cmap='Grays_r')
    ax[1].imshow(image2_crop > threshold, cmap='Grays_r')
    plt.show()

    ct1, ct1_center, ct2, ct2_center = match_reflectors(image1_crop, image2_crop, refl_brightness=refl_brightness[i], blur=True)
    if type(ct1)!=np.ndarray:
        continue
    
    fig, ax = plt.subplots(1, 2, figsize=(10,5))
    ax[0].imshow(image1_crop, cmap='Grays_r')
    ax[0].plot(ct1[:,0], ct1[:,1], '-m')
    ax[0].plot(ct1_center[0], ct1_center[1], '*m')
    ax[0].set_title('Image 1')

    ax[1].imshow(image2_crop, cmap='Grays_r')
    ax[1].plot(ct2[:,0], ct2[:,1], '-m')
    ax[1].plot(ct2_center[0], ct2_center[1], '*m')
    ax[1].set_title('Image 2')

    ax[0].axis('off')
    ax[1].axis('off')
    plt.show()

   


In [ ]:
ch = 'ch11'
image1_files = sorted(glob(os.path.join(out_folder, '*.tiff')))
image1_file = [x for x in image1_files if ch in os.path.basename(x)][0]
image1 = cv2.imread(image1_file, cv2.IMREAD_GRAYSCALE)
image2_files = sorted(glob(os.path.join(data_folder, '..', 'camera_calibration', 'images20251021', '*.tiff')))
image2_file = [x for x in image2_files if ch in os.path.basename(x)][0]
image2 = cv2.imread(image2_file, cv2.IMREAD_GRAYSCALE)


%matplotlib widget

plt.figure(figsize=(8,8))
plt.imshow(image1, cmap='Grays_r')
plt.show()